# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
import os
sys.path.append('../../src')
os.environ['BEN_HOME'] = "../.."

import logging

# Set logging level to suppress warnings
logging.getLogger().setLevel(logging.ERROR)
# Just disables the warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
from nn.models import Models
from bots import BotBid
from bidding import bidding
from util import hand_to_str
from deck52 import random_deal
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)

In [2]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
models.search_threshold = 0.07
models.double_dummy_calculator = True
# Filter to remove hands not consistent with the bidding. Used during play
models.bid_accept_play_threshold = 0.01
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies
#sampler.use_distance = False
#sampler.bidding_threshold_sampling = 0.25


Instructions for updating:
non-resource variables are not supported in the long term
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_same-5556000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_same-5556000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/Jack/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/jack/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/jack/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/jack/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/jack/decl-1000000
INFO:tensorflow:Restoring parameters from ..\Models/jack/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/jack/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/

### Bidding through an entire auction for all four hands

Now we'll take a random deal and let the bot bid for each hand. We assume North deals and nobody is vulnerable.

This script was used to detect the reason for some strange bidding. But the error is fixed, so this should look normal now

In [3]:

hands = ["AJ.KJ9.A9764.AJ8","T853.AT65.Q8.T52", "K976.Q874.KT.K74", "Q42.32.J532.Q963"]
#hands = ['AKQJTx.AQTx.Ax.x', 'xxx.Kx.KT8x.KJxx', 'x.J98xxxx.9.QTxx', '98x..QJxxxx.A98x']

In [4]:
# North is dealer
dealer = 0
bidder_bots = [BotBid([False, True], hand, models, sampler, i, dealer, False) for i, hand in enumerate(hands)]

In [5]:
auction = [] 

turn_i = 0 # whose turn is it to bid
step = 0

while not bidding.auction_over(auction):
    bid = bidder_bots[turn_i].bid(auction)
    #if step== 5:
    #     bid.bid='X'
    #if step== 1:
    #     bid.bid='PASS'
    print(bid.to_dict())
    auction.append(bid.bid)
    turn_i = (turn_i + 1) % 4  # next player's turn
    step += 1
print(auction)

{'bid': '1D', 'who': 'NN', 'candidates': [{'call': '1D', 'insta_score': 0.961}], 'hcp': [7.5, 8.1, 6.9], 'shape': [3.8, 3.2, 2.6, 3.2, 3.6, 3.2, 2.8, 3.2, 3.7, 3.2, 2.7, 3.3]}
{'bid': 'PASS', 'who': 'NN', 'candidates': [{'call': 'PASS', 'insta_score': 1.0}], 'hcp': [10.5, 10.0, 13.5], 'shape': [3.0, 3.2, 3.2, 3.5, 3.2, 3.3, 2.8, 3.6, 2.5, 2.6, 5.0, 2.8]}
{'bid': '1H', 'who': 'NN', 'candidates': [{'call': '1H', 'insta_score': 1.0}], 'hcp': [9.0, 13.2, 6.0], 'shape': [3.2, 3.3, 2.7, 3.7, 2.6, 2.7, 4.9, 2.7, 3.0, 3.2, 3.1, 3.7]}
{'bid': 'PASS', 'who': 'NN', 'candidates': [{'call': 'PASS', 'insta_score': 1.0}], 'hcp': [14.3, 7.4, 12.5], 'shape': [3.4, 2.8, 4.4, 2.5, 3.8, 3.1, 2.8, 3.4, 3.2, 4.8, 2.1, 2.8]}
{'bid': '3N', 'who': 'NN', 'candidates': [{'call': '3N', 'insta_score': 0.992}], 'hcp': [6.1, 11.0, 6.1], 'shape': [4.0, 2.5, 3.0, 3.5, 3.1, 4.6, 2.4, 2.8, 4.0, 2.6, 2.9, 3.5]}
{'bid': 'PASS', 'who': 'NN', 'candidates': [{'call': 'PASS', 'insta_score': 1.0}], 'hcp': [10.2, 5.3, 18.1], 's

Loaded lib dds.dll


{'bid': 'PASS', 'who': 'Simulation', 'quality': 'Good', 'candidates': [{'call': 'PASS', 'insta_score': 0.143, 'expected_score': 471, 'adjustment': 7}, {'call': '4N', 'insta_score': 0.569, 'expected_score': 439, 'adjustment': 28}, {'call': '6N', 'insta_score': 0.259, 'expected_score': 437, 'adjustment': 13}], 'samples': ['AT8.KT.AQJ8x.AJ8 Jxxx.Axxx.xx.xxx K9xx.Q8xx.KT.Kxx Qx.J9x.9xxx.QT9x 0.99622 \n 1D-P-1H-P-3N-P-4N-P-5D-P-6N-P-P-P (12, 11.4, 11.1)  \n 1D-P-1H-P-3N-P-6N-P-P-P (12, 11.4, 11.1)  \n 1D-P-1H-P-3N-P-P-P (12, 11.4, 11.1) ', 'Txx.AK.AQJ8x.AJx AJ8x.9xx.9xx.QT9 K9xx.Q8xx.KT.Kxx Qx.JTxx.xxx.8xxx 0.99619 \n 1D-P-1H-P-3N-P-4N-P-5D-P-6N-P-P-P (11, 11.0, 10.9)  \n 1D-P-1H-P-3N-P-6N-P-P-P (11, 11.0, 10.9)  \n 1D-P-1H-P-3N-P-P-P (11, 11.0, 10.9) ', 'Ax.AJ9.AJ9xx.AJT Txx.KT.Q8xx.98xx K9xx.Q8xx.KT.Kxx QJ8x.xxxx.xx.Qxx 0.99617 \n 1D-P-1H-P-3N-P-4N-P-5D-P-6N-P-P-P (11, 11.2, 10.9)  \n 1D-P-1H-P-3N-P-6N-P-P-P (11, 11.2, 10.9)  \n 1D-P-1H-P-3N-P-P-P (11, 11.2, 10.9) ', 'AQx.Kx.AJ9xx.AJx Txx